In [1]:
!pip install pycaret
!pip install jinja
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached MarkupSafe-2.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Jinja-1.2.tar.gz (252 kB)
  Using cached Jinja-1.1.tar.gz (237 kB)
  Using cached Jinja-1.0.tar.gz (120 kB)
  Using cached Jinja-0.9.tar.gz (55 kB)
  Using cached Jinja-0.8.tar.gz (55 kB)
ERROR: Could not find a version that satisfies the requirement jin

In [2]:
from pycaret.classification import*
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/data/235687_시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_err = pd.read_csv(path + 'train_err_data.csv')
id_error = train_err[['user_id', 'errtype']].values
error = np.zeros((15000,42))
for person_idx, err in tqdm(id_error):
    # person_idx -10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000, err - 1] += 1

train_prob = pd.read_csv(path+'train_problem_data.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1

train = pd.DataFrame(data=error)
train['problem'] = problem
del error, problem

clf = setup(data = train, target = 'problem')

,Description,Value
0,session_id,6492
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 43)"
5,Missing Values,False
6,Numeric Features,42
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [14]:
best_4 = compare_models(sort='Accuracy', n_select=4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7757,0.7979,0.4463,0.7845,0.5686,0.4322,0.4637,2.218
lightgbm,Light Gradient Boosting Machine,0.7744,0.7976,0.4836,0.7474,0.5870,0.4414,0.4613,0.350
et,Extra Trees Classifier,0.7724,0.7932,0.4612,0.7581,0.5731,0.4302,0.4551,1.884
rf,Random Forest Classifier,0.7705,0.7916,0.4494,0.7604,0.5647,0.4225,0.4496,2.020
ada,Ada Boost Classifier,0.7642,0.7823,0.4420,0.7429,0.5539,0.4074,0.4329,0.611
lr,Logistic Regression,0.7522,0.7270,0.3667,0.7616,0.4947,0.3564,0.3983,1.853
qda,Quadratic Discriminant Analysis,0.7466,0.7249,0.3687,0.7354,0.4902,0.3459,0.3827,0.059
lda,Linear Discriminant Analysis,0.7446,0.7325,0.3141,0.7873,0.4486,0.3206,0.3780,0.107
ridge,Ridge Classifier,0.7436,0.0000,0.3060,0.7930,0.4412,0.3152,0.3755,0.040
nb,Naive Bayes,0.7103,0.7146,0.2310,0.6847,0.3432,0.2139,0.2660,0.040


In [15]:
# 개를 골라 블렌딩
blended = blend_models(estimator_list = best_4, fold = 4, method ='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7867,0.8033,0.4483,0.8298,0.5821,0.4555,0.4944
1,0.7703,0.7949,0.4690,0.7432,0.5751,0.4285,0.4498
2,0.7817,0.8096,0.4575,0.7976,0.5814,0.4481,0.4798
3,0.7774,0.7960,0.4609,0.7771,0.5786,0.4405,0.4683
Mean,0.7790,0.8010,0.4589,0.7869,0.5793,0.4432,0.4731
Std,0.0060,0.0059,0.0074,0.0315,0.0028,0.0100,0.0163


In [16]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7849,0.8147,0.4684,0.8165,0.5953,0.4631,0.4963


In [10]:
final_model = finalize_model(blended)

In [ ]:
test_err  = pd.read_csv(path+'test_err_data.csv')
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

In [ ]:
predictions = predict_model(final_model, data = test)

In [ ]:
#pyacaret에서는 score이 label을 맞출 확률이기 때문에 output을 제출 양식에 맞게 바꿔줍니다.
# x = []
# for i in range(len(predictions['Score'])):
#     if predictions['Label'][i] == '1.0':
#         x.append(predictions['Score'][i])
#     else:
#         x.append(1-predictions['Score'][i])

In [ ]:
# sample_submission = pd.read_csv(path+'sample_submission.csv')
# sample_submission['problem'] = x
# sample_submission.to_csv("AutoML.csv", index=False)